In [1]:
import torch

/home/marc/miniconda3/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pytorch_lightning as pl
from pytorch_lightning import loggers as pl_loggers

In [3]:
import os
import torch
print(os.getcwd())

from torchrbpnet.data import tfrecord_to_dataloader, dummy_dataloader
from torchrbpnet.data import datasets

dataloader = torch.utils.data.DataLoader(datasets.TFIterableDataset('../example/data.matrix/head.tfrecord', batch_size=2, shuffle=100), batch_size=None)

/home/marc/PhD/TorchRBPNet/notebooks


In [4]:
from torchrbpnet import layers

In [5]:
layers.StemConv1D

torchrbpnet.layers.stems.StemConv1D

In [6]:
import gin
import gin.torch.external_configurables

@gin.configurable()
def train(network, max_epochs, optimizer):
    pass

gin.parse_config_file('../configs/hyparam.search/config.render-001.gin')

b = layers.RNAConv1dBody()

In [7]:
b(torch.rand(1, 4, 1001)).shape

/home/marc/miniconda3/envs/torch/lib/python3.10/site-packages/torch/nn/modules/conv.py:309: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:895.)
  return F.conv1d(input, weight, bias, self.stride,


torch.Size([1, 128, 1001])

In [8]:
# %%
import torch
import torch.nn as nn
import pytorch_lightning as pl

import torchmetrics
from torchrbpnet.losses import MultinomialNLLLossFromLogits
from torchrbpnet.metrics import MultinomialNLLFromLogits #BatchedPCC
from torchrbpnet.networks import MultiRBPNet

class Model(pl.LightningModule):
    def __init__(self, network, metrics=None, optimizer=torch.optim.Adam):
        super().__init__()
        self.network = network
        self.loss_fn = MultinomialNLLLossFromLogits()
        
        # metrics
        if metrics is None:
            self.metrics = nn.ModuleDict({})
        else:
            self.metrics = nn.ModuleDict(metrics)
        
        # optimizer
        self.optimizer_cls = optimizer
    
    def forward(self, *args, **kwargs):
        return self.network(*args, **kwargs)

    def configure_optimizers(self):
        optimizer = self.optimizer_cls(self.parameters())
        return optimizer

    def training_step(self, batch, batch_idx, **kwargs):
        inputs, y = batch
        y = y['total']
        y_pred = self.forward(inputs)
        loss = self.loss_fn(y, y_pred, dim=-2)
        self.compute_and_log_metics(y_pred, y, partition='train')
        return loss
    
    def training_epoch_end(self, *args, **kwargs):
        self._reset_metrics()

    def validation_epoch_end(self, *args, **kwargs):
        self._reset_metrics()

    def validation_step(self, batch, batch_idx):
        inputs, y = batch
        y = y['total']
        y_pred = self.forward(inputs)
        self.compute_and_log_metics(y_pred, y, partition='val')
    
    def compute_and_log_metics(self, y_pred, y, partition=None):
        on_step = False
        if partition == 'train':
            on_step = True

        for name, metric in self.metrics.items():
            metric(y_pred, y)
            self.log(f'{partition}/{name}', metric.compute(), on_step=on_step, on_epoch=True, prog_bar=False)
    
    def _reset_metrics(self):
        for metric in self.metrics.values():
            metric.reset()

model = Model(network=MultiRBPNet(n_tasks=223))

In [9]:
model

Model(
  (network): MultiRBPNet(
    (body): RNAConv1dBody(
      (stem): StemConv1D(
        (conv1d): Conv1d(4, 128, kernel_size=(12,), stride=(1,), padding=same)
        (batch_norm): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): ReLU()
      )
      (tower): Sequential(
        (0): Sequential(
          (0): ResConv1DBlock(
            (conv1d): Conv1d(128, 256, kernel_size=(3,), stride=(1,), padding=same)
            (batch_norm): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (act): SiLU()
            (dropout): Dropout1d(p=0.15, inplace=False)
            (linear_upsample): Conv1d(128, 256, kernel_size=(1,), stride=(1,), bias=False)
          )
          (1): ResConv1DBlock(
            (conv1d): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=same)
            (batch_norm): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (act): Si

In [15]:
mods = [x for x in model.modules()]

In [24]:
mods[-5].weight.shape

torch.Size([128, 256, 1])

In [10]:
import datetime
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, RichProgressBar, LearningRateMonitor

root_log_dir = f'logs/{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}'
loggers = [
    pl_loggers.TensorBoardLogger(root_log_dir+'/tensorboard', name='', version='', log_graph=True),
    # pl_loggers.CSVLogger(root_log_dir+'/tensorboard', name='', version=''),
]

checkpoint_callback = ModelCheckpoint(dirpath=f'{root_log_dir}/checkpoints', every_n_epochs=1, save_last=True)

early_stop_callback = EarlyStopping(monitor="val/loss", min_delta=0.00, patience=3, verbose=False, mode="min")

bar = RichProgressBar()

trainer = pl.Trainer(default_root_dir=root_log_dir, max_epochs=2, logger=loggers, callbacks=[checkpoint_callback, early_stop_callback, LearningRateMonitor('step', log_momentum=True)])
trainer.fit(model=model, train_dataloaders=dataloader, val_dataloaders=dataloader)
torch.save(model.network, 'test.pt')

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/marc/miniconda3/envs/torch/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:176: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(

  | Name    | Type                         | Params
---------------------------------------------------------
0 | network | MultiRBPNet                  | 1.6 M 
1 | loss_fn | MultinomialNLLLossFromLogits | 0     
2 | metrics | ModuleDict                   | 0     
---------------------------------------------------------
1.6 M     Trainable params
0         Non-trainable params
1.6 M     Total params
6.439     Total estimated model params size (MB)
/home/marc/miniconda3/envs/torch/lib/python3.10/site-packages/pytorch_lightning/loggers/tensorboard.py:188: UserWarning: Could not log computati

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/home/marc/miniconda3/envs/torch/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


RuntimeError: mat1 and mat2 shapes cannot be multiplied (256x1000 and 256x223)

In [ ]:
model.optimizers()